In [ ]:
import json
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from googletrans import Translator
from langdetect import detect, DetectorFactory
import numpy as np
import time
import ast



In [ ]:
data = ""

with open('summaries.json', 'r', encoding="utf8") as f:
    data = json.load(f)

In [ ]:
# create dataframe
ids = []
conversations = []

for value in data:
    id = value["id"]
    conversation = value["summary"]
    
    conversations.append(conversation)
    ids.append(id)


dataframe_info = {"id": ids, "summaries": conversations }

df = pd.DataFrame(dataframe_info)
df.head()

In [ ]:
print(len(df))

In [ ]:
corpus = df["summaries"]

In [ ]:
cached = stopwords.words('english')

def preprocess_text(text):
    text = str(text)
    # remove code
    a=re.compile("<code>.*?</code>")
    text = a.sub('', text)
    # remove angle brackets and all characters inside angle bracket
    a=re.compile("<.*?>")
    text = a.sub('', text)
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # convert to lowercase
    text = text.lower()
    # remove code, user, ai, conversation
    text = text.replace("code", "")
    text = text.replace("user", "")
    text = text.replace("ai", "")
    text = text.replace("conversation", "")
    text = text.replace("security", "")
    # tokenize the text
    tokens = word_tokenize(text)
    # remove stopwords
    tokens = [word for word in tokens if word not in cached]
    return tokens

# preprocess all text
# preprocessed_text = [' '.join(preprocess_text(document)) for document in corpus]

preprocessed_text = []
i = 0
for document in corpus:
    print(i)
    preprocessed_document = ' '.join(preprocess_text(document))
    preprocessed_text.append(preprocessed_document)
    i += 1





In [ ]:
df["preprocessed_text"] = preprocessed_text

In [ ]:
df.head()

In [ ]:
corpus = df["preprocessed_text"]

In [ ]:
# tokenize the documents
tokenized_documents = [doc.split() for doc in corpus]

# create a dictionary representation of the documents
dictionary = Dictionary(tokenized_documents)

# convert document into bag-of-words format
corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]


In [ ]:
lda_model = LdaModel(corpus, num_topics=200, id2word=dictionary, random_state=42)


In [ ]:
topics = lda_model.print_topics(num_topics=200)
for topic in topics:
    print(topic)

In [ ]:
# setup: get the model's topics in their native ordering...
all_topics = lda_model.print_topics(num_topics=300)
# ...then create a empty list per topic to collect the docs:
docs_per_topic = [[] for _ in all_topics]

# now, for every doc...
for doc_id, doc_bow in enumerate(corpus):
    # ...get its topics...
    doc_topics = lda_model.get_document_topics(doc_bow)
    # ...& for each of its topics...
    for topic_id, score in doc_topics:
        # ...add the doc_id & its score to the topic's doc list
        docs_per_topic[topic_id].append((doc_id, score))



In [ ]:
topic_ids = [7]
doc_ids = set()

for topic_id in topic_ids:
    for (doc_id, score) in docs_per_topic[topic_id]:
        doc_ids.add(doc_id)


print(len(doc_ids))




In [ ]:
ids = []
conversations = []


for doc_id in doc_ids:
    id = df["id"][doc_id]
    ids.append(id)

    conversation = df["text"][doc_id]
    conversations.append(conversation)



sp_df = pd.DataFrame()
sp_df["id"] = ids
sp_df["conversation"] = conversations


In [ ]:
sp_df.to_csv('sp_data.csv', index=False)
